## Import Packages

In [1]:
!pip install tensorflow_decision_forests

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import datasets, model_selection 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics import accuracy_score, classification_report
import tensorflow_decision_forests as tfdf
import pandas as pd

## Import dataset

In [1]:
iris = pd.read_csv("/kaggle/input/iris/Iris.csv")
iris

## EDA

In [1]:
iris.info()

In [1]:
iris.describe()

As we can see This is a balanced dataset.

In [1]:
iris.groupby("Species")["Species"].count()

## Train Validation Split
I will apply train validation split to this dataset. This is a very small dataset, so it will be better if we do train validation split by their species.

In [1]:
validation_split = 0.2

In [1]:
species = sorted(list(iris["Species"].unique()))
species

In [1]:
train_features, test_features = None, None
for item in species:
    data = iris[iris.Species == item]
    train_subset, test_subset = train_test_split(data, test_size=validation_split)
    if train_features is None or test_features is None:
        train_features = train_subset
        test_features = test_subset
    else:
        train_features = pd.concat([train_features, train_subset])
        test_features = pd.concat([test_features, test_subset])
print(train_features.shape, test_features.shape)
train_features = sklearn.utils.shuffle(train_features)
train_features.pop("Id")
test_features.pop("Id")
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_features, label="Species")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_features, label="Species")

## Model Development

In [1]:
model = tfdf.keras.RandomForestModel(num_trees=30)
model.compile(metrics=["accuracy"])
model.fit(train_ds, validation_data=test_ds)

## Prediction

In [1]:
test_labels = model.predict(test_ds)
test_labels[:10], test_labels.shape